<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_31.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')
target = pd.read_csv('/content/target.csv', encoding = 'cp949')

In [ ]:
!pip install autokeras

In [5]:
import tensorflow as tf
import autokeras as ak

Using TensorFlow backend


In [6]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

current_year = datetime.now().year

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

# 선택한 행의 인덱스를 사용하여 삭제
data = data.drop(selected_rows.index)

In [7]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22995 entries, 0 to 23010
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      22995 non-null  datetime64[ns]
 1   최고기온    22995 non-null  float64       
 2   최저기온    22995 non-null  float64       
 3   일교차     22995 non-null  float64       
 4   강수량     22995 non-null  float64       
 5   평균습도    22995 non-null  float64       
 6   평균풍속    22995 non-null  float64       
 7   일조합     22995 non-null  float64       
 8   일사합     22995 non-null  float64       
 9   일조율     22995 non-null  float64       
 10  평균기온    22995 non-null  float64       
 11  월       22995 non-null  int64         
 12  계절      22995 non-null  object        
dtypes: datetime64[ns](1), float64(10), int64(1), object(1)
memory usage: 2.5+ MB


In [9]:
# 입력 특성과 출력 정의
x_train = data.drop(columns=['평균기온'])  # 입력 특성
y_train = data['평균기온']  # 출력 정답

# AutoML 모델 정의
regressor = ak.StructuredDataRegressor(max_trials=1)  # max_trials는 조정 가능

# 모델 훈련
regressor.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 00m 47s]
val_loss: 18.861038208007812

Best val_loss So Far: 18.861038208007812
Total elapsed time: 00h 00m 47s
Epoch 1/10
719/719 [==============================] - 3s 3ms/step - loss: 63.0697 - mean_squared_error: 63.0697
Epoch 2/10
719/719 [==============================] - 3s 4ms/step - loss: 27.0125 - mean_squared_error: 27.0125
Epoch 3/10
719/719 [==============================] - 2s 3ms/step - loss: 23.6715 - mean_squared_error: 23.6715
Epoch 4/10
719/719 [==============================] - 2s 2ms/step - loss: 21.2694 - mean_squared_error: 21.2694
Epoch 5/10
719/719 [==============================] - 2s 3ms/step - loss: 19.0783 - mean_squared_error: 19.0783
Epoch 6/10
719/719 [==============================] - 3s 4ms/step - loss: 17.2218 - mean_squared_error: 17.2218
Epoch 7/10
719/719 [==============================] - 2s 2ms/step - loss: 15.6630 - mean_squared_error: 15.6630
Epoch 8/10
719/719 [==============================] - 2s 3ms/step - loss: 14.2968 

In [10]:
# AutoKeras 모델 확인
model = regressor.export_model()  # AutoKeras 모델을 내보내서 TensorFlow 모델로 변환
model.summary()  # TensorFlow 모델의 구조 확인

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12)]              0         
                                                                 
 multi_category_encoding (M  (None, 12)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 12)                25        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                416       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056  

In [11]:
reg_pred = regressor.predict(x_train)

719/719 [==============================] - 2s 3ms/step


In [12]:
reg_pred

array([[-0.18395013],
       [ 1.273874  ],
       [-0.5117442 ],
       ...,
       [-3.534882  ],
       [-0.7947194 ],
       [-2.3408792 ]], dtype=float32)

In [13]:
len(reg_pred)

22995

In [16]:
import numpy as np

# 주어진 배열
input_array = np.array(reg_pred)  # 22995개의 행

# 365행을 갖는 62개의 열로 변환
output_array = input_array.reshape((63, 365))

# 결과 출력
print(output_array)

[[ -0.18395013   1.273874    -0.5117442  ...  -9.951786   -10.002059
  -10.816078  ]
 [ -7.977419    -2.5471232   -1.091387   ...  -2.5020301    3.2233264
    3.7211173 ]
 [  3.2900908   -3.3180823    0.25730544 ...   3.5340087   -2.127716
   -6.2721367 ]
 ...
 [ -3.4054973    1.6245344   -1.049896   ...   1.0456235   -8.541038
   -8.3586445 ]
 [ -1.2169287   -5.702276    -4.058614   ...   1.320512    -3.2801087
   -2.970232  ]
 [ -4.6700783   -0.17465562  -0.6342165  ...  -3.534882    -0.7947194
   -2.3408792 ]]


In [22]:
# 각 열별로 평균 계산하여 365개의 행으로 만들기
averages = np.mean(output_array, axis=0)
result_array = averages.reshape((365, 1))

# 결과 출력
print(result_array)

[[-1.08813250e+00]
 [-1.16219687e+00]
 [-1.43978584e+00]
 [-2.27469993e+00]
 [-2.19756842e+00]
 [-1.24598551e+00]
 [-1.02808785e+00]
 [-9.30350482e-01]
 [-1.63795543e+00]
 [-1.87684882e+00]
 [-1.94383371e+00]
 [-2.21204758e+00]
 [-2.71196127e+00]
 [-2.85589099e+00]
 [-2.01386023e+00]
 [-1.90940225e+00]
 [-1.66031682e+00]
 [-1.85171056e+00]
 [-2.47643137e+00]
 [-1.56403506e+00]
 [-1.74122536e+00]
 [-1.67210186e+00]
 [-2.60564113e+00]
 [-2.33477306e+00]
 [-1.43313348e+00]
 [-1.70017564e+00]
 [-1.61492908e+00]
 [-1.43310392e+00]
 [-1.24026561e+00]
 [-1.99889743e+00]
 [-1.60676694e+00]
 [-2.27135277e+00]
 [-2.43417168e+00]
 [-2.07606602e+00]
 [-1.58456838e+00]
 [-1.62435699e+00]
 [-1.16797638e+00]
 [-1.21641731e+00]
 [-1.22548616e+00]
 [-9.86445546e-01]
 [-6.08069122e-01]
 [-7.55711436e-01]
 [-5.47325730e-01]
 [-6.09528832e-03]
 [-1.47713318e-01]
 [-4.18047488e-01]
 [-4.77444410e-01]
 [-4.66508746e-01]
 [-8.39841727e-04]
 [ 2.32564569e-01]
 [ 3.44712466e-01]
 [-1.67413384e-01]
 [ 4.2238688

In [23]:
submission['평균기온'] = result_array[:358]
print(submission.head(30))

            일시      평균기온
0   2023-01-01 -1.088133
1   2023-01-02 -1.162197
2   2023-01-03 -1.439786
3   2023-01-04 -2.274700
4   2023-01-05 -2.197568
5   2023-01-06 -1.245986
6   2023-01-07 -1.028088
7   2023-01-08 -0.930350
8   2023-01-09 -1.637955
9   2023-01-10 -1.876849
10  2023-01-11 -1.943834
11  2023-01-12 -2.212048
12  2023-01-13 -2.711961
13  2023-01-14 -2.855891
14  2023-01-15 -2.013860
15  2023-01-16 -1.909402
16  2023-01-17 -1.660317
17  2023-01-18 -1.851711
18  2023-01-19 -2.476431
19  2023-01-20 -1.564035
20  2023-01-21 -1.741225
21  2023-01-22 -1.672102
22  2023-01-23 -2.605641
23  2023-01-24 -2.334773
24  2023-01-25 -1.433133
25  2023-01-26 -1.700176
26  2023-01-27 -1.614929
27  2023-01-28 -1.433104
28  2023-01-29 -1.240266
29  2023-01-30 -1.998897


In [24]:
submission.to_csv("DEAGYUL_25_submission.csv", index=False)

print(submission.head())

           일시      평균기온
0  2023-01-01 -1.088133
1  2023-01-02 -1.162197
2  2023-01-03 -1.439786
3  2023-01-04 -2.274700
4  2023-01-05 -2.197568
